In [11]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [12]:
BASE_URL = 'https://www.basketball-reference.com/'

draft_2012 = 'https://www.basketball-reference.com/draft/NBA_2012.html'
draft_2013 = 'https://www.basketball-reference.com/draft/NBA_2013.html'
draft_2014 = 'https://www.basketball-reference.com/draft/NBA_2014.html'
draft_2015 = 'https://www.basketball-reference.com/draft/NBA_2015.html'
draft_2016 = 'https://www.basketball-reference.com/draft/NBA_2016.html'
draft_2017 = 'https://www.basketball-reference.com/draft/NBA_2017.html'
draft_2018 = 'https://www.basketball-reference.com/draft/NBA_2018.html'

draft_2021 = 'https://www.basketball-reference.com/draft/NBA_2021.html'

draft_2011 = 'https://www.basketball-reference.com/draft/NBA_2011.html'

# list_url = [ draft_2012, draft_2013, draft_2014, 
#             draft_2015, draft_2016, draft_2017, draft_2018]
# list_url = [draft_2021]
list_url = [draft_2011]

div class="table_container is_setup" id="div_stats"
    table class="sortable stats_table now_sortable" id="stats" data-cols-to-freeze="1,2,3"
        tbody
            tr data-row="0" class="rowSum"
                td data-stat="pick_overall"      => ***PICK RANK***
                td data-stat="player"
                    a href="***URL STATS NBA***" => ***NOM***
                td data-stat="college_name"
                    a                            => ***College name***

In [13]:
players = []

for url in list_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find('div',class_="table_container").find('table', class_='sortable stats_table').find('tbody')

    for player in table.find_all('tr'):
        college = None #initiate college variable as not every player comes from an american university
        player_name = None
        for td in player.find_all('td'):
            if td['data-stat'] == "player":
                player_url = BASE_URL+td.find('a').get('href')
                player_name = td.find('a').string
            elif td['data-stat'] == "pick_overall":
                pick_rank = td.find('a').string
            elif td['data-stat'] == "college_name" and td.find('a'):
                college = td.find('a').string
        if player_name:
            players.append({'player': player_name, 'pick rank':pick_rank , 'year': url[47:51], 'college':college, 'url': player_url})


In [14]:
players[0]['url']

'https://www.basketball-reference.com//players/i/irvinky01.html'

In [15]:
def convert_height(feet,inches):
    h_ft = int(feet)
    h_inch = int(inches)

    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 1)
    return h_cm

In [16]:
for player in players:

    player_response = requests.get(player['url'])
    player_soup = BeautifulSoup(player_response.content, "html.parser")
    
    if player_soup.find('span', itemprop="height"):
        feet, inches = player_soup.find('span', itemprop="height").string.split('-')
        player['height (cm)']=convert_height(feet,inches)

        player['weight (lb)']=player_soup.find('span', itemprop="weight").string[0:3]
    
    for li in player_soup.find_all('li'):
        if li.find('a') and li.find('a').string=='College Basketball at Sports-Reference.com':
            uni_url_string = li.find('a').get('href')
            player_id = uni_url_string.split('/')[-1][:-5]
            player['uni_url']=uni_url_string
            player['player_id']=player_id
            break

In [17]:
players_df = pd.DataFrame(players)
players_df

,player,pick rank,year,college,url,height (cm),weight (lb),uni_url,player_id
0,Kyrie Irving,1,2011,Duke,https://www.basketball-reference.com//players/...,188.0,195,https://www.sports-reference.com/cbb/players/k...,kyrie-irving-1
1,Derrick Williams,2,2011,Arizona,https://www.basketball-reference.com//players/...,203.2,240,https://www.sports-reference.com/cbb/players/d...,derrick-williams-2
2,Enes Kanter,3,2011,None,https://www.basketball-reference.com//players/...,208.3,250,NaN,NaN
3,Tristan Thompson,4,2011,Texas,https://www.basketball-reference.com//players/...,205.7,254,https://www.sports-reference.com/cbb/players/t...,tristan-thompson-2
4,Jonas Valančiūnas,5,2011,None,https://www.basketball-reference.com//players/...,210.8,265,NaN,NaN
5,Jan Veselý,6,2011,None,https://www.basketball-reference.com//players/...,210.8,240,NaN,NaN
6,Bismack Biyombo,7,2011,None,https://www.basketball-reference.com//players/...,203.2,255,NaN,NaN
7,Brandon Knight,8,2011,Kentucky,https://www.basketball-reference.com//players/...,188.0,182,https://www.sports-reference.com/cbb/players/b...,brandon-knight-1
8,Kemba Walker,9,2011,UConn,https://www.basketball-reference.com//players/...,182.9,184,https://www.sports-reference.com/cbb/players/k...,kemba-walker-1
9,Jimmer Fredette,10,2011,BYU,https://www.basketball-reference.com//players/...,188.0,195,https://www.sports-reference.com/cbb/players/j...,jimmer-fredette-1


In [18]:
# players_df.to_csv('../raw_data/players.csv', index=False)

In [19]:
# players_df.to_csv('../raw_data/draft_2021.csv', index=False)

In [20]:
players_df.to_csv('../raw_data/draft_2011.csv', index=False)